In [ ]:
import nest
import os
import anndata
import scipy
import sklearn.metrics
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import squidpy as sq
import networkx as nx
from pathlib import Path
import time
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rc

import leidenalg as la
import igraph as ig
from scipy.sparse import coo_array
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.colors as colors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset_dir = os.path.expanduser("/Users/ben/Library/CloudStorage/Dropbox/data/ms/axel_datasets/")
dataset_file = os.path.join(dataset_dir, "parigi22_merged.h5ad")
adata = anndata.read_h5ad(dataset_file)

In [ ]:
image_save_dir = "images/"
def save_fig(fig, name):
    fig.savefig(os.path.join(image_save_dir, name), dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
adata1 = adata[adata.obs['condition'] == 'WT'].copy()
adata1.uns['spatial'] = {'V19S23-097_A1': adata1.uns['spatial']['V19S23-097_A1']}

In [ ]:
adata2 = adata[adata.obs['condition'] == 'DSS'].copy()
adata2.uns['spatial'] = {'V19S23-097_B1': adata2.uns['spatial']['V19S23-097_B1']}

In [ ]:
neighbor_eps = 105
min_samples = 3
hotspot_min_size = 15

In [ ]:
for adata in [adata1, adata2]:
    n = num_hotspot_genes = nest.compute_gene_hotspots(adata, verbose=True, core_only=False, log=True,
                                               eps=neighbor_eps, min_samples=min_samples, min_size=hotspot_min_size)
    nest.coexpression_hotspots(adata, threshold=0.50, min_size=5, cutoff=0.3, min_genes=5, resolution=1.0)
    nest.compute_multi_boundaries(adata, 10, 0.001)

In [ ]:
nest.plot.multi_hotspots(adata2, inds=[9], legend_ncol=2)

In [ ]:
nest.plot.plot_similarity_map(adata=adata1, adata_ref=adata2, idx=36, spot_size=100, vmax=1)

In [ ]:
arr = []
inds = list(range(len(adata2.uns['multi_hotspots'])))
for idx in inds:
    v = nest.similarity_map(adata1, idx=idx, adata_ref=adata2)
    arr.append(np.mean(v))

df = pd.DataFrame(index=inds, data={'sim': arr})

In [ ]:
#df.sort_values(by='sim')

In [ ]:
nest.plot.multi_hotspots(adata2, inds=[36], legend_ncol=2)

In [ ]:
nest.plot.spatial(adata1, color=[v[0][9:] for v in adata2.uns['multi_hotspots']['17']], spot_size=100)

In [ ]:
nest.plot.spatial(adata2, color=[v[0][9:] for v in adata2.uns['multi_hotspots']['48']], spot_size=100)

In [ ]:
fig, ax = plt.subplots()
nest.plot.multi_hotspots(adata1, ax=ax)
#save_fig(fig=fig, name="multi_hotspots_WT.png")

In [ ]:
fig, ax = plt.subplots()
nest.plot.multi_hotspots(adata2, ax=ax)
#save_fig(fig=fig, name="multi_hotspots_DSS.png")

In [ ]:
for idx in [0, 1, 9, 17]:
    fig, ax = plt.subplots(figsize=[2, 2])
    nest.plot.plot_similarity_map(adata=adata2, adata_ref=adata2, idx=idx, spot_size=100, vmax=1, color_map="Blues", ax=ax, show=False, linewidth=0.5,
                                  linecolor="red")
    ax.collections[0].colorbar.remove()
    #save_fig(fig, f"sim_{idx}_intra.png")

    fig, ax = plt.subplots(figsize=[2, 2])
    nest.plot.plot_similarity_map(adata=adata1, adata_ref=adata2, idx=idx, spot_size=100, vmax=1, color_map="Blues", ax=ax, show=False, linewidth=0.5,
                                  linecolor="red")
    ax.collections[0].colorbar.remove()
    #save_fig(fig, f"sim_{idx}_inter.png")
    #break

In [ ]:
fig, ax = plt.subplots()
nest.plot.spatial(adata1, color="Krt13", spot_size=100, ax=ax, show=False, color_map="Blues")
#save_fig(fig, f"krt13_wt.png")

fig, ax = plt.subplots()
nest.plot.spatial(adata2, color="Krt13", spot_size=100, ax=ax, show=False, color_map="Blues")
#save_fig(fig, f"krt13_dss.png")